# Jalkapallo Fantasy -data

## Tarkoituksena on hakea FPL-pelin dataa ja johtaa siitä hyödyllisiä johtopäätöksiä. FPL on peli, jossa valitaan jalkapallopelaajia ja saadaan pisteitä riippuen siitä, kuinka hyvin he pärjäävät otteluissa

### Tätä tehdessä käytetty apuna erilaisia ohjeita netistä.

In [1]:
import requests
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
## Haetaan data
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)

## Muunnetaan data JSONiksi
json = r.json()

## Katsotaan avaimia
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [3]:
## Rakennetaan dataframet:

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [4]:
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,100.0,100.0,58822,0,0,-3,3,0,2,0.5,...,575,158,488,190,410,154,406,154,0.00000,0.00000
1,NaN,NaN,84450,0,0,0,0,1,3,3.5,...,195,111,107,40,48,20,99,35,1.05817,0.44554
2,50.0,100.0,153256,-1,1,-4,4,1,3,0.3,...,605,308,359,166,383,165,205,62,0.81081,0.00000
3,NaN,NaN,156074,0,0,-3,3,0,2,1.0,...,584,165,322,109,414,158,431,164,0.00000,0.00000
4,75.0,100.0,167199,-1,1,-3,3,0,3,2.0,...,306,185,158,65,124,57,257,84,1.03448,0.58190


In [5]:
## Valitaan tärkeimmät datat:
slim_elements_df = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]


In [6]:
slim_elements_df.head()

## Datassa näkyy nyt pelaaja ja tärkeimmät datat kuten pelatut minuutit, hinta, arvo, kokonaispisteet.

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points
0,Alves Soares,1,2,0.1,42,28,6746,0.5,2
1,Xhaka,1,3,2.9,50,1616,804305,15.8,79
2,Elneny,1,3,0.8,41,111,194328,1.5,6
3,Holding,1,2,0.1,42,16,6800,1.4,6
4,Partey,1,3,0.5,47,1392,96136,11.3,53


In [7]:
## Selvitetään pelaajan pelipaikka
slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)
slim_elements_df.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position
0,Alves Soares,1,2,0.1,42,28,6746,0.5,2,Defender
1,Xhaka,1,3,2.9,50,1616,804305,15.8,79,Midfielder
2,Elneny,1,3,0.8,41,111,194328,1.5,6,Midfielder
3,Holding,1,2,0.1,42,16,6800,1.4,6,Defender
4,Partey,1,3,0.5,47,1392,96136,11.3,53,Midfielder


In [8]:
## Selvitetään joukkueen nimi
slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)


In [9]:
## Muunnetaan Value-taulukon arvoja sringistä floatiksi, 
# jotta niitä voi järjestää helpommin
slim_elements_df['value'] = slim_elements_df.value_season.astype(float)

In [10]:
slim_elements_df.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,value
0,Alves Soares,Arsenal,2,0.1,42,28,6746,0.5,2,Defender,0.5
1,Xhaka,Arsenal,3,2.9,50,1616,804305,15.8,79,Midfielder,15.8
2,Elneny,Arsenal,3,0.8,41,111,194328,1.5,6,Midfielder,1.5
3,Holding,Arsenal,2,0.1,42,16,6800,1.4,6,Defender,1.4
4,Partey,Arsenal,3,0.5,47,1392,96136,11.3,53,Midfielder,11.3


In [12]:
## Nyt voimme järjestää pelaajat esimerkiksi. arvon mukaan
# (eniten pisteitä hintaan nähden)
slim_elements_df.sort_values('value', ascending=False).head(10)

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,value
510,Trippier,Newcastle,2,68.9,61,1774,6397953,23.1,141,Defender,23.1
110,Raya Martin,Brentford,1,8.8,47,1800,1249977,19.4,91,Goalkeeper,19.4
522,Almirón Rejala,Newcastle,3,38.1,58,1656,5455968,19.3,112,Midfielder,19.3
530,Botman,Newcastle,2,6.0,45,1507,836077,18.9,85,Defender,18.9
511,Burn,Newcastle,2,3.7,46,1674,505290,18.9,87,Defender,18.9
529,Pope,Newcastle,1,24.8,55,1800,3457242,18.7,103,Goalkeeper,18.7
519,Schär,Newcastle,2,15.9,52,1694,2744075,18.7,97,Defender,18.7
8,White,Arsenal,2,13.8,47,1509,1954666,17.9,84,Defender,17.9
557,Henderson,Nott'm Forest,1,8.0,47,1620,1416558,17.4,82,Goalkeeper,17.4
317,Hoelgebaum Pereira,Fulham,3,20.4,45,1728,2798917,17.3,78,Midfielder,17.3


In [13]:
## Tarkastellaan sitten, ketkä pelaajat ovat saaaneet eniten pisteitä ylipäätään
slim_elements_df.sort_values('total_points', ascending=False).head(10)

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,value
452,Haaland,Man City,4,85.3,122,1545,5515868,13.9,169,Forward,13.9
510,Trippier,Newcastle,2,68.9,61,1774,6397953,23.1,141,Defender,23.1
617,Kane,Spurs,4,36.1,117,1876,4888295,12.0,140,Forward,12.0
5,Ødegaard,Arsenal,3,32.3,69,1522,4300644,17.0,117,Midfielder,17.0
522,Almirón Rejala,Newcastle,3,38.1,58,1656,5455968,19.3,112,Midfielder,19.3
109,Toney,Brentford,4,30.7,77,1611,7643134,14.3,110,Forward,14.3
11,Saka,Arsenal,3,23.9,82,1624,3274004,13.3,109,Midfielder,13.3
476,Rashford,Man Utd,3,49.6,72,1616,7021563,15.0,108,Midfielder,15.0
437,De Bruyne,Man City,3,37.3,125,1566,5646737,8.6,107,Midfielder,8.6
408,Salah,Liverpool,3,25.7,128,1687,3575380,8.0,103,Midfielder,8.0


In [15]:
# Tästä huomaamme, että kolme parasta pelaajaa hinta/laatu-suhteeltaan olisi:
  #  1. Trippier, 2. Raya, 3. Almiron
# Huomaamme myös, että kolme parasta pelaajaa ylipäätään ovat:
  # 1. Haaland, 2. Trippier, 3. Kane


In [21]:
## Kokeillaan Pivot-taulukolla tarkastella pelaajien arvo per pelipaikka
slim_elements_df.pivot_table(index="position", values="value", aggfunc=np.mean)

,value
position,
Defender,4.662656
Forward,3.667778
Goalkeeper,4.350685
Midfielder,4.419805


In [22]:
## Ongelmaksi muodostuu pelaajat, joilla on 0 pelattua minuuttia, ja jotka sen takia tiputtavat arvoja. Poistetaan pelaajat, joilla 0 minuuttia

In [23]:
slim_elements_df = slim_elements_df.loc[slim_elements_df.value > 0]

In [24]:
pivot = slim_elements_df.pivot_table(index='position',values='value',aggfunc=np.mean).reset_index()

In [25]:
pivot.sort_values('value',ascending=False)

,position,value
2,Goalkeeper,11.342857
0,Defender,6.459770
3,Midfielder,5.918696
1,Forward,5.329032


In [26]:
## Seuraavaksi voidaan Pivot-taulukon avulla selvittää, mitkä joukkueet ovat olleet parhaita arvoltaan
team_pivot = slim_elements_df.pivot_table(index="team", values="value", aggfunc=np.mean).reset_index()

In [27]:
team_pivot.sort_values("value", ascending=False)

,team,value
14,Newcastle,8.926087
0,Arsenal,8.525000
12,Man City,7.571429
6,Crystal Palace,7.280000
13,Man Utd,7.265217
4,Brighton,7.227273
3,Brentford,7.188000
10,Leicester,7.022727
1,Aston Villa,6.763636
8,Fulham,6.381481


In [ ]:
## Tästä huomataan, että Newcastle ja Arsenal olleet parhaat joukkueet arvoltaan valita pelaajia. Southampton ja Liverpool huonoimmat
